In [66]:
import requests
import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime, timezone

In [81]:
def get_weather_data(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather_data = {
            "Timestamp": datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S'), 
            "City": city,
            "Temperature (°C)": data["main"]["temp"],
            "Humidity (%)": data["main"]["humidity"],
            "Description": data["weather"][0]["description"],
            # Use timezone-aware UTC datetime
            "Time": datetime.fromtimestamp(data["dt"], tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'),
            # Sunrise and Sunset as timezone-aware datetime
            "Sunrise": datetime.fromtimestamp(data["sys"]["sunrise"], tz=timezone.utc).strftime('%H:%M:%S'),
            "Sunset": datetime.fromtimestamp(data["sys"]["sunset"], tz=timezone.utc).strftime('%H:%M:%S'),
            "Wind Speed (m/s)": data["wind"]["speed"],
            "Weather Icon": data["weather"][0]["icon"]
        }
        return weather_data
    else:
        print("Error: Unable to fetch weather data.")
        return None

In [89]:
def connect_to_gsheet(json_path, sheet_name):
    # Define the scope
    scope = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
    
    # Authenticate using service account credentials
    creds = Credentials.from_service_account_file(json_path, scopes=scope)
    client = gspread.authorize(creds)

    try:
        # Open the Google Sheets document by its name
        sheet = client.open(sheet_name).sheet1
        print("Connected to sheet:", sheet_name)

        # Check if headers are already added (first row)
        header = sheet.row_values(1)  # Read the first row (header)
        
        if not header:  # If the header is empty, append the headers
            print("Headers not found. Appending headers...")
            sheet.append_row([
                "Timestamp", "City", "Temperature (°C)", "Humidity (%)", "Description", 
                "Time", "Wind Speed (m/s)", "Sunrise", "Sunset", "Weather Icon"
            ])
        else:
            print("Headers already exist.")

        return sheet
    
    except Exception as e:
        print(f"Error: {e}")
        return None  # Return None if there's an error

# Usage example: provide the path to your credentials JSON and the name of the sheet
sheet = connect_to_gsheet(r"C:\data\data-with-stephen-457108-a8e720844e4b.json", 'WeatherData')

if sheet:
    print("Connection to Google Sheets established!")
else:
    print("Failed to connect to Google Sheets.")

Connected to sheet: WeatherData
Headers already exist.
Connection to Google Sheets established!


In [83]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
# Replace with your actual JSON path
json_key_path = "C:/data/data-with-stephen-457108-a8e720844e4b.json"  # Update with your file path
creds = Credentials.from_service_account_file(json_key_path, scopes=scope)# Connect to Google Sheets

client = gspread.authorize(creds)
sheet = client.open("WeatherData").sheet1

In [84]:
while True:
    city = input("🔍 Enter a city to get weather (or type 'exit' to quit): ")
    if city.lower() == 'exit':
        break

    data = get_weather_data(city, api_key)
    if data:
        for key, value in data.items():
            print(f"{key}: {value}")

        # Append the weather data to the Google Sheet
        row = [
            data["Timestamp"],
            data["City"],
            data["Temperature (°C)"],
            data["Humidity (%)"],
            data["Description"],
            data["Time"],
            data["Wind Speed (m/s)"],
            data["Sunrise"],
            data["Sunset"],
            data["Weather Icon"]
        ]
        sheet.append_row(row)
        print("📤 Sent to Google Sheets!")

🔍 Enter a city to get weather (or type 'exit' to quit):  Lagos


Timestamp: 2025-04-24 06:01:40
City: Lagos
Temperature (°C): 26
Humidity (%): 88
Description: overcast clouds
Time: 2025-04-24 05:53:48
Sunrise: 05:33:27
Sunset: 17:52:27
Wind Speed (m/s): 0.77
Weather Icon: 04d
📤 Sent to Google Sheets!


🔍 Enter a city to get weather (or type 'exit' to quit):  London


Timestamp: 2025-04-24 06:02:14
City: London
Temperature (°C): 9.22
Humidity (%): 87
Description: overcast clouds
Time: 2025-04-24 05:59:54
Sunrise: 04:45:38
Sunset: 19:11:17
Wind Speed (m/s): 1.54
Weather Icon: 04d
📤 Sent to Google Sheets!


🔍 Enter a city to get weather (or type 'exit' to quit):  England


...

🔍 Enter a city to get weather (or type 'exit' to quit):  Ajah


Timestamp: 2025-04-24 06:03:28
City: Ajah
Temperature (°C): 27.99
Humidity (%): 79
Description: overcast clouds
Time: 2025-04-24 06:03:48
Sunrise: 05:34:18
Sunset: 17:53:06
Wind Speed (m/s): 2.58
Weather Icon: 04d
📤 Sent to Google Sheets!


🔍 Enter a city to get weather (or type 'exit' to quit):  Kenya


Timestamp: 2025-04-24 06:03:37
City: Kenya
Temperature (°C): 25.59
Humidity (%): 62
Description: broken clouds
Time: 2025-04-24 06:02:15
Sunrise: 03:21:39
Sunset: 15:30:17
Wind Speed (m/s): 5.76
Weather Icon: 04d
📤 Sent to Google Sheets!


🔍 Enter a city to get weather (or type 'exit' to quit):  exit


In [85]:
print(sheet.get_all_values())

[['Timestamp', 'City', 'Temperature (°C)', 'Humidity (%)', 'Description', 'Time', 'Wind Speed (m/s)', 'Sunrise', 'Sunset', 'Weather Icon'], ['2025-04-24 06:01:40', 'Lagos', '26', '88', 'overcast clouds', '2025-04-24 05:53:48', '0.77', '05:33:27', '17:52:27', '04d'], ['2025-04-24 06:02:14', 'London', '9.22', '87', 'overcast clouds', '2025-04-24 05:59:54', '1.54', '04:45:38', '19:11:17', '04d'], ['2025-04-24 06:02:20', 'England', '18.78', '93', 'overcast clouds', '2025-04-24 06:02:40', '2.96', '11:24:59', '00:46:35', '04n'], ['2025-04-24 06:02:25', 'Dubai', '29.96', '51', 'clear sky', '2025-04-24 05:53:58', '4.63', '01:48:12', '14:45:18', '01d'], ['2025-04-24 06:02:34', 'Hungary', '16.05', '81', 'overcast clouds', '2025-04-24 06:02:54', '2.95', '03:35:54', '17:40:01', '04d'], ['2025-04-24 06:02:39', 'California', '9.97', '84', 'clear sky', '2025-04-24 06:02:59', '0', '10:17:31', '23:50:22', '01n'], ['2025-04-24 06:02:47', 'Connecticut', '10.04', '75', 'clear sky', '2025-04-24 06:03:07', 

In [86]:
import pandas as pd

# Load the data from Google Sheets
import gspread
from google.oauth2.service_account import Credentials

In [87]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("C:/data/data-with-stephen-457108-a8e720844e4b.json", scopes=scope)
client = gspread.authorize(creds)
sheet = client.open("WeatherData").sheet1

data = pd.DataFrame(sheet.get_all_records())
print(data.head())

             Timestamp     City  Temperature (°C)  Humidity (%)  \
0  2025-04-24 06:01:40    Lagos             26.00            88   
1  2025-04-24 06:02:14   London              9.22            87   
2  2025-04-24 06:02:20  England             18.78            93   
3  2025-04-24 06:02:25    Dubai             29.96            51   
4  2025-04-24 06:02:34  Hungary             16.05            81   

       Description                 Time  Wind Speed (m/s)   Sunrise    Sunset  \
0  overcast clouds  2025-04-24 05:53:48              0.77  05:33:27  17:52:27   
1  overcast clouds  2025-04-24 05:59:54              1.54  04:45:38  19:11:17   
2  overcast clouds  2025-04-24 06:02:40              2.96  11:24:59  00:46:35   
3        clear sky  2025-04-24 05:53:58              4.63  01:48:12  14:45:18   
4  overcast clouds  2025-04-24 06:02:54              2.95  03:35:54  17:40:01   

  Weather Icon  
0          04d  
1          04d  
2          04n  
3          01d  
4          04d  


In [88]:
data = data.drop_duplicates()  # Removes duplicates
data = data.fillna('N/A')  # Fills empty spots with 'N/A'